Develop unit tests for aerotech Ensemble class.
For mocker advice: https://stackoverflow.com/questions/36888882/how-to-mock-a-socket-object-via-the-mock-library

For unittest example: https://github.com/robmarkcole/TrasportAPI-HASS/blob/master/RC_Transport_API%20%2021-7-2017.ipynb

In [1]:
import sys
print(sys.version)
import time

from aerotech import Ensemble
import unittest
from unittest import mock
import socket

3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]


## Investigate mocker
Mocker could be used for unittest in the absence of the Ensmble emulator

In [2]:
# test class

import python_utils as utils

class TCPSocket:
    def __init__(self):
        self.buf = ''

    def __enter__(self):
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def connect(self, host, port):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((host, port))

    def close(self):
        self.sock.close()

    def send(self, data):
        self.sock.send(data)

    def sendall(self, data):
        self.sock.sendall(data)

    # For best match with hardware and network realities,
    # the value of limit should be a relatively small power of 2, for example, 4096
    def recv_some(self, limit=4096):
        return self.sock.recv(limit)

In [3]:
(1024).to_bytes(2, byteorder='big')#.decode("utf-8") 

b'\x04\x00'

In [4]:
class TestSensor(unittest.TestCase):
    
    def test_class(self): 
        mock_bytes = (1024).to_bytes(2, byteorder='big')
        count = 4096
        with mock.patch('socket.socket') as mock_socket:
            mock_socket.return_value.recv.return_value = "test_return"
            t = TCPSocket()
            t.connect('example.com', 12345)  # t.sock is a mock object, not a Socket
            print(t.recv_some())
            
            self.assertEqual(t.recv_some(), "test_return")
            t.sock.connect.assert_called_with(('example.com', 12345))
            
unittest.main(argv=['ignored', '-v'], exit=False)

test_class (__main__.TestSensor) ... 

test_return


ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## Ensemble test
Test with emulator running rather than mocker

In [5]:
ip = 'localhost'
port = 8000

my_ensemble = Ensemble(ip, port)
my_ensemble.connect()

my_ensemble.move(1, 10, 5.5)

my_ensemble.get_positions()['X']
my_ensemble.close()

In [7]:
class Test(unittest.TestCase):
    def setUp(self):
        self.my_ensemble = Ensemble(ip, port)
        self.my_ensemble.connect()
        self.my_ensemble.move(1, 10, 5.5) 
        time.sleep(2)
        
    def test_ensemble(self):              
        self.assertEqual(self.my_ensemble.get_positions()['X'], 1.0)
        
    def tearDown(self):
        self.my_ensemble.close()
            
unittest.main(argv=['ignored', '-v'], exit=False)

test_ensemble (__main__.Test) ... ok
test_class (__main__.TestSensor) ... 

test_return


ok

----------------------------------------------------------------------
Ran 2 tests in 2.009s

OK
